## Создание продвинутых ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Yandex Assistant API, RAG и Function Calling.

Для начала, установим Yandex Cloud ML SDK. В идеальном мире, вы сделаете вот так:

In [ ]:
%pip install --upgrade --quiet yandex-cloud-ml-sdk

В нашем случае мы установим самую последнюю версию SDK:

In [ ]:
%pip install --quiet flit
%pip install --quiet -I git+https://github.com/yandex-cloud/yandex-cloud-ml-sdk.git@assistants_fc#egg=yandex-cloud-ml-sdk

Также необходимо обновить некоторые библиотеки:

In [ ]:
%pip install --upgrade --quiet pydantic

**ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

И ещё пара полезных функций на будущее:

In [1]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

Для работы с языковыми моделями нам понадобится ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere.

Создадим модель последней версии YandexGPT 5 и убедимся, что она кое-что знает про вина:

In [2]:
import os
from yandex_cloud_ml_sdk import YCloudML

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]

sdk = YCloudML(folder_id=folder_id, auth=api_key)

# Раскомментируйте, если хотите подробнее смотреть, что делает SDK
#sdk.setup_default_logging(log_level='DEBUG')

model = sdk.models.completions("yandexgpt", model_version="rc")

In [3]:
printx(model.run("Какое вино можно пить со стейком?").text)

Выбор вина к стейку зависит от степени прожарки мяса, его вида и дополнительных ингредиентов, которые присутствуют в блюде. Вот несколько примеров вин, которые могут хорошо сочетаться со стейком:

1. **Каберне Совиньон** — это крепкое вино с насыщенным вкусом и ароматом, которое может хорошо дополнить вкус жареного мяса.
2. **Мерло** — ещё один сорт винограда, который часто используется для производства вин, подходящих к стейку. Мерло обычно имеет более мягкий вкус и менее терпкий, чем Каберне Совиньон.
3. **Шираз (Сира)** — это вино с насыщенным вкусом и ароматом тёмных фруктов, которое может хорошо сочетаться с жирным мясом, таким как стейк из мраморной говядины.
4. **Мальбек** — аргентинский сорт винограда, который даёт вина с насыщенным вкусом и ароматом ягод. Мальбек может хорошо дополнить вкус стейка, приготовленного на гриле.
5. **Темпранильо** — испанский сорт винограда, из которого производят вина с насыщенным вкусом и ароматом красных фруктов. Темпранильо может хорошо сочетаться с различными видами стейков.

Важно помнить, что выбор вина — это дело вкуса, и то, что подходит одному человеку, может не подойти другому. Рекомендуется экспериментировать с разными винами и находить те, которые лучше всего соответствуют вашим предпочтениям.

## Assistant API

Для ведения беседы с моделью с сохранением контекста диалога используем Assistants API. Объект `thread` будет отвечать за сохранение контекста, а `assistant` - за все основные установки, связанные с работой ассистента.

In [4]:
def create_thread():
    return sdk.threads.create(ttl_days=1, expiration_policy="static")

def create_assistant(model, tools=None):
    kwargs = {}
    if tools and len(tools) > 0:
        kwargs = {"tools": tools}
    return sdk.assistants.create(
        model, ttl_days=1, expiration_policy="since_last_active", **kwargs
    )

Создадим простого ассистента и беседу:

In [5]:
thread = create_thread()
assistant = create_assistant(model)

assistant.update(
    instruction="""Ты - опытный сомелье, задача которого - консультировать пользователя в
    вопросах выбора вина."""
)

thread.write("Привет! Какое вино посоветуете?")

run = assistant.run(thread)
result = run.wait()

printx(result.text)

Здравствуйте! Чтобы подобрать вино, которое вам понравится, мне нужно знать несколько деталей:

1. Какой тип вина вы предпочитаете: красное, белое, розовое, игристое или десертное?
2. Какой стиль вина вам интересен: лёгкое и свежее или более насыщенное и крепкое?
3. Есть ли у вас предпочтения по вкусам и ароматам? Например, вы любите фруктовые, ягодные, цветочные или более сложные ноты?
4. В каком ценовом диапазоне вы ищете вино?
5. С какими блюдами вы планируете сочетать вино? Это поможет подобрать наиболее подходящий вариант.

Чем больше информации вы предоставите, тем точнее я смогу порекомендовать вино, которое будет соответствовать вашим предпочтениям.

In [6]:
thread.write("Я буду есть стейк!")

run = assistant.run(thread)
result = run.wait()

printx(result.text)

Отлично! Для стейка я могу порекомендовать несколько вариантов красного вина:

1. Каберне Совиньон — это насыщенный и структурированный сорт вина с нотами тёмных ягод, дуба и специй, который прекрасно дополнит вкус мяса.
2. Мерло — более мягкий и фруктовый вариант с нотами вишни, сливы и шоколада, который также хорошо подойдёт к стейку.
3. Шираз (Сира) — насыщенное и пряное вино с нотами тёмного фрукта, специй и дуба, которое может стать отличным дополнением к стейку, приготовленному на гриле.
4. Мальбек — это вино с насыщенным вкусом и ароматом тёмных фруктов, ванили и шоколада, которое хорошо подойдёт к стейку средней прожарки.

Это лишь несколько примеров вин, которые могут хорошо сочетаться со стейком. Если у вас есть предпочтения по стилю или вкусу, пожалуйста, сообщите мне, и я смогу дать более конкретные рекомендации. Также важно учитывать ценовой диапазон, в котором вы ищете вино.

В итоге в переписке `thread` содержится вся история сообщений:

In [7]:
for msg in list(thread)[::-1]:
    printx(f"**{msg.author.role}:** {msg.text}")

**USER:** Привет! Какое вино посоветуете?

**ASSISTANT:** Здравствуйте! Чтобы подобрать вино, которое вам понравится, мне нужно знать несколько деталей:

1. Какой тип вина вы предпочитаете: красное, белое, розовое, игристое или десертное?
2. Какой стиль вина вам интересен: лёгкое и свежее или более насыщенное и крепкое?
3. Есть ли у вас предпочтения по вкусам и ароматам? Например, вы любите фруктовые, ягодные, цветочные или более сложные ноты?
4. В каком ценовом диапазоне вы ищете вино?
5. С какими блюдами вы планируете сочетать вино? Это поможет подобрать наиболее подходящий вариант.

Чем больше информации вы предоставите, тем точнее я смогу порекомендовать вино, которое будет соответствовать вашим предпочтениям.

**USER:** Я буду есть стейк!

**ASSISTANT:** Отлично! Для стейка я могу порекомендовать несколько вариантов красного вина:

1. Каберне Совиньон — это насыщенный и структурированный сорт вина с нотами тёмных ягод, дуба и специй, который прекрасно дополнит вкус мяса.
2. Мерло — более мягкий и фруктовый вариант с нотами вишни, сливы и шоколада, который также хорошо подойдёт к стейку.
3. Шираз (Сира) — насыщенное и пряное вино с нотами тёмного фрукта, специй и дуба, которое может стать отличным дополнением к стейку, приготовленному на гриле.
4. Мальбек — это вино с насыщенным вкусом и ароматом тёмных фруктов, ванили и шоколада, которое хорошо подойдёт к стейку средней прожарки.

Это лишь несколько примеров вин, которые могут хорошо сочетаться со стейком. Если у вас есть предпочтения по стилю или вкусу, пожалуйста, сообщите мне, и я смогу дать более конкретные рекомендации. Также важно учитывать ценовой диапазон, в котором вы ищете вино.

После использования переписку и ассистента можно удалить.

In [8]:
thread.delete()
assistant.delete()

## Добавляем RAG

Для RAG будем использовать текстовую базу знаний по винам и винным регионам, которая хранится в виде множества файлов в директориях `data/wines` и `data/regions`. Пройдёмся по этим файлам и посмотрим на их длину в токенах.

In [9]:
from glob import glob
from tqdm.auto import tqdm
import pandas as pd

def get_token_count(filename):
    with open(filename, "r", encoding="utf8") as f:
        return len(model.tokenize(f.read()))

def get_file_len(filename):
    with open(filename, encoding="utf-8") as f:
        l = len(f.read())
    return l

d = [
    {
        "File": fn,
        "Tokens": get_token_count(fn),
        "Chars": get_file_len(fn),
        "Category": fn.split("/")[1],
    }
    for fn in glob("data/*/*.md")
    if fn.count("/") == 2
]

df = pd.DataFrame(d)
df

,File,Tokens,Chars,Category
0,data/regions/Абруццо.md,499,2022,regions
1,data/regions/Азорские острова.md,409,1809,regions
2,data/regions/Аконкагуа.md,278,1182,regions
3,data/regions/Алентежу.md,320,1216,regions
4,data/regions/Апулия.md,489,1918,regions
...,...,...,...,...
125,data/wines/Совиньон блан.md,634,2578,wines
126,data/wines/Темпранильо.md,637,2323,wines
127,data/wines/Цвайгельт.md,663,2620,wines
128,data/wines/Шардоне.md,567,2407,wines


Посмотрим на среднюю, мин и макс длину фрагментов:

In [10]:
df.groupby("Category").agg({"Tokens": ("min", "mean", "max")})

Tokens                 
            min        mean  max
Category                        
regions     153  423.340000  683
wines       264  551.366667  664

Мы видим, что фрагменты не превышают 700 токенов, и это значит, что нам не придётся прибегать к какой-либо стратегии чанкования. Это идеальная ситуация, когда текстовая база знаний разбита вручную на небольшие фрагменты текста.

## Загружаем файлы в облако

Чтобы RAG мог осущетвлять поиск по фрагментам файлов, нам необходимо построить индекс, а перед этим - загрузить все файлы в облако.

In [11]:
def upload_file(filename):
    return sdk.files.upload(filename, ttl_days=1, expiration_policy="static")

df["Uploaded"] = df["File"].apply(upload_file)

## Строим индекс

Для индексации файлов можно применять следующие стратегии:
* Поиск по эмбеддингам (векторный поиск) - по всем фрагментам текста вычисляются эмбеддинги, и в процессе поиска осуществляется векторный поиск между эмбеддингом запроса и эмбеддингом фрагмента. Это позволяет осуществлять семантический поиск
* Поиск по ключевым словам
* Гибридный поиск, в котором различными способами объединяются результаты поиска по ключевым словам и вектороного поиска.

Также в процессе создания индекса мы указываем **стратегию чанкования** текстов, но в нашем случае она не будет применяться.

> Поскольку есть ограничение на 100 добавляемых в индекс файлов, то будем добавлять фрагменты по винам и по регионам по-очереди.

In [12]:
from yandex_cloud_ml_sdk.search_indexes import (
    StaticIndexChunkingStrategy,
    HybridSearchIndexType,
    ReciprocalRankFusionIndexCombinationStrategy,
)

op = sdk.search_indexes.create_deferred(
    df[df["Category"] == "wines"]["Uploaded"],
    index_type=HybridSearchIndexType(
        chunking_strategy=StaticIndexChunkingStrategy(
            max_chunk_size_tokens=1000, chunk_overlap_tokens=100
        ),
        combination_strategy=ReciprocalRankFusionIndexCombinationStrategy(),
    ),
)
index = op.wait()

In [13]:
op = index.add_files_deferred(df[df["Category"]=="regions"]["Uploaded"])
xfiles = op.wait()

## Собираем RAG-ассистента

Теперь собираем собственно ассистента, который будет использовать RAG. Для этого определяем **инструмент** (tool) для поиска в нашем индексе, и указываем его при создании ассистента. Также важно задать хорошую инструкцию для ассистента (системный промпт): 

In [14]:
search_tool = sdk.tools.search_index(index)

assistant = create_assistant(model, tools=[search_tool])
thread = create_thread()

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде. Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций. Если что-то непонятно, то лучше уточни информацию
у пользователя.
"""

_ = assistant.update(instruction=instruction)

In [15]:
thread.write("Какое вино подходит к стейку?")
run = assistant.run(thread)

result = run.wait()
printx(result.text)

К стейку подойдут следующие вина:

1. Из региона Франшхук в Южной Африке — красные вина из сортов каберне совиньон, шираз, мерло, а также пинотаж.
2. Из долины Маллеко в Чили — красное сухое вино.
3. Из штата Орегон в США — пино нуар.

Посмотрим, из каких источников был получен этот ответ:

In [16]:
def print_citations(result):
    for citation in result.citations:
        for source in citation.sources:
            if source.type != "filechunk":
                continue
            print("------------------------")
            printx(source.parts[0])

print_citations(result)

------------------------


## Сицилия 

Сицилия
Сицилия — регион на юге Италии. В его состав входит одноименный и самый большой в Средиземном море остров и несколько более мелких островов рядом с ним. На восточном побережье Сицилии возвышается Этна — самый высокий действующий вулкан в Европе. Помимо итальянского в регионе говорят на сицилийском языке. Административный центр Сицилии — Палермо. Это самый южный из винодельческих регионов Италии. Здесь около 120 тысяч га виноградников. С начала нулевых на Сицилии бурно развивается производство вулканических вин. Их производят из винограда, выросшего у подножия Этны. Пепел самого высокого действующего вулкана Европы богат микроэлементами, поэтому местные почвы очень плодородны. И, главное, в таких условиях нет шанса у филлоксеры. К наиболее значимым местным сортам относят красный неро д'авола. Также на Сицилии популярны красные гренаш, перриконе и ночера. Из белых стоит отметить катаратто — его смешивают с сортами грилло и инзолия и производят легендарное крепленое вино марсала. Также популярные былые греканико, александрийский мускат и типичный итальянский треббьяно. Король вулканических красных сортов — нерелло маскалезе с ароматами спелой вишни, пряностей, табака и трав. У подножия Этны успешно выращивают белый автохтонный сорт карриканте. Он обладает спокойной минеральностью, цитрусово-травянистым тонами и вкусом, который ассоциируют с чистым воздухом гор. Легендарная сицилийская крепленая марсала — нежная, яркая и фруктовая. По популярности с ней сопоставим разве что москато ди пантеллерия (Moscato di Pantelleria) из александрийского муската. Из красных вин особого внимания заслуживает черасуоло ди виттория (Cerasuolo di Vittoria) — сицилийское вино наивысшего качества (DOCG). Ну и, конечно, легендарные вулканические вина. Сорт нерелло маскалезе отдает вину высокие танины, кислотность, ягодные ноты и минеральные оттенки. Хрустящие вина из белого карриканте выделяются свежестью, минеральностью, лёгкой солоноватостью. В их аромате слышны яблоки, цитрусы, анис. Крепленая марсала прекрасна в любом виде: сладкая подходит на роль аперитива, полусухая играет с фруктовыми салатами или сицилийскими канноли, а сухая — с теплым мясным салатом или супом. Сладкие сицилийские мускаты отлично подойдут в пару к козьим сырам и пирогам типа нежного киша с сыром. Красное черасуоло ди виттория гармонирует с тушеной говядиной, жареной курицей или мясом на вертеле. И, пожалуй, все вина стоит попробовать в сочетании с местными сырными шариками аранчини с разными начинками. Идеальной парой для белого вина из вулканического винограда карриканте станут креветки, равиоли с травами или ризотто.

------------------------


## Франшхук 

Франшхук
Франшхук — крупная винодельческая провинция Южной Африки. Территориально относится к региону Стелленбош, расположена в 75 км от Кейптауна — столицы ЮАР. Деревню основали беглые французы в 1688 году. Получив земельные наделы, они разбили виноградники и наладили производство напитков. Сегодня Франшхук называют винной столицей ЮАР. На карте провинции можно насчитать 11 крупных винодельческих хозяйств. Деревня Франшхук известна необычным терруаром: она расположена в продолговатой долине и с трех сторон окружена высокими горами, которые защищают виноградники от ветра, излишней влаги зимой и палящего солнца летом. При этом четвертая сторона долины открыта для ветров Атлантики. Климат здесь умеренный, а температура воздуха ниже, чем в соседних областях. Виноград созревает медленнее, чем на открытой африканской местности, что делает вино более свежим и кислотным. В Франшхуке наиболее распространены известные европейские сорта, завезенные французами в XVII веке. Здесь выращивают белые шенен блан, семильон, совиньон блан и шардоне. Среди красных популярны каберне совиньон, шираз и мерло. Значительная часть плантаций занята под пинотаж — автохтонный сорт, ставший визитной карточкой страны. Линейка традиционных вин из европейских сортов в ЮАР открывается совершенно по-новому. Сухие гранитные почвы придают напиткам явную минеральность. Белые вина приобретают золотистый оттенок и аромат южных фруктов, красные — насыщенный цвет и аромат. Оттенок может варьироваться от ярко-рубинового до глубокого фиолетового. Их часто сравнивают с бургундскими напитками. С 1992 года в долине Франшхук производят игристые вина премиального качества из шардоне и пино нуар по классическому методу. Белые вина Франшхука — прекрасный аперитив. Они подходят к легким сырам, закускам из гусиного паштета и курицы. Среди горячих блюд стоит выбрать индейку, запеченную белую рыбу с овощным гарниром или рагу. Красные южноафриканские сорта хорошо гармонируют с перчеными стейками из говядины и баранины, а также твердыми, выдержанными сырами. Пинотаж при этом считается универсальным вином. Ему подходят разнообразные блюда от изысканного филе миньон до лазаньи и пиццы с морепродуктами.

------------------------


## Сур 

Сур
Сур — винодельческий субрегион в Чили, один из самых южных в стране. В его границах выделяют долины Итата, Био-Био, Маллеко. Чилийские вина олицетворяют стиль Нового Света, для них характерен мягкий фруктовый вкус. По одной из версий, первые лозы в Чили завезли с территории Перу в XVI веке. Виноделием занимались христианские миссионеры, выращивали преимущественно красный сорт мишн, белые делали из москателя. До середины XIX века здесь были популярны сладкие вина. В долине Итата первые лозы высадили в середине XVI века после создания порта Консепсьон. Почвы здесь с хорошим дренажем, климат средиземноморский с продолжительным сухим сезоном. В Био-Био и Маллеко прохладно из-за воздушных потоков с океана. Виноград успевает вызревать благодаря большому количеству теплых дней. В Суре выращивают совиньон блан, шардоне, рислинг, вионье, гевюрцтраминер, мускат. Сур известен столовыми винами. Климат в регионе умеренный, близкий к погодным условиям Франции, поэтому местные виноделы стали выпускать и более элегантные напитки. Условия в долинах Био-Био и Итата отлично подходят для культивирования шардоне, совиньон блан, рислинга, из которых делают белые вина. Здесь хорошо растет пино нуар, из него получают красные вина. Розовые вина из долины Итата хороши со свежими фруктами, десертами, сырами, ветчиной; белые сладкие — с десертами, фруктами. Белое полусухое из Био-Био составит отличную пару с блюдами индийской и азиатской кухонь, курице, овощам. Красное сухое из Маллеко сочетается с запеченными овощами, свининой, стейком на гриле.

------------------------


## Орегон 

Орегон
Орегон — четвертый по объемам производства вина штат в США. Самый популярный сорт винограда, который выращивают здесь, — пино нуар. Вина из этого сорта отличает высокая кислотность и низкая танинность. Выращивать виноград в Орегоне начали еще в 1840-х, но датой зарождения серьезного виноделия называют 1961 год, когда любители бургундского пино нуар Ричард Соммер и Дэвид Летт решили производить похожее вино в Америке. Для этого был выбран Орегон — прохладный климат и разнообразие почв позволили добиться успеха. Уже в 1979 пино нуар из Орегона вошло в десятку вин в бургундском стиле на французской винной Олимпиаде. Его отличают более яркие ноты фруктов и более низкая кислотность. Прохладный климат Орегона хорошо подходит для выращивания таких сортов винограда, как рислинг, шардоне и гаме, но визитной карточкой штата считается пино нуар, который растет в долине Уилламетт. Менее популярны мерло, каберне совиньон, зинфандель. В Орегоне производят сухие, полусухие, а также игристые и десертные вина. Пино нуар из Орегона будет хорошо сочетаться со стейком из лосося, запеченной птицей и красным мясом, например, говядиной по-бургундски.

------------------------


## Гевюрцтраминер 

Гевюрцтраминер
Гевюрцтраминер (нем. Gewurztraminer) — сорт белого винного винограда с розовой кожицей родом из Эльзаса. Перевод названия — пряный траминер — указывает на родство с сортом траминер. Чтобы сбалансировать избыточное количество сахара и сохранить кислотность, гевюрцу подходит климат без засухи и сильной жары. Лоза рано зацветает, но плодоносит поздно, восприимчива к болезням и заморозкам. Виноград выращивают в Эльзасе, северной Италии, Германии, Австрии, Австралии, США и России. История гевюрца начинается с древнего винограда траминер с зеленой кожицей, получившего свое название от деревни Трамин в Южном Тироле, немецкоязычной провинции на севере Италии. А возраст старейшего виноградника гевюрца в немецком Пфальце насчитывает более 400 лет. Поскольку виноград трудно выращивать, в XX веке ученые-виноделы из Германии неоднократно пытались скрещивать гевюрцтраминер с другими сортами, но ни один из гибридов не оказался успешным. Из гевюрцтраминера производят тихие сухие, сладкие и десертные вина, чаще — моносортовые. Такое вино наполнено нотами цветов, имбиря и тропических фруктов: личи, абрикоса, персика, ананаса, дыни. Как правило, гевюрцтраминеры отличаются повышенным содержанием алкоголя (13,5-15%), иногда резкостью, маслянистостью и горечью в финале. Природные низкая кислотность и сладость винограда делают вино идеальным спутником пряных и пикантных блюд тайской, индийской, ближневосточной кухонь. Сухой или полусухой гевюрцтраминер можно подавать с красным тайским карри, пикантной лапшой Пад Тай, собой в соусе Том Ям с креветками, семгой с имбирем в соевом соусе, желтым карри из индейки. А на десерт — сорбет из манго, фило с заварным кремом, пахлава с розовой водой и бельгийские вафли с фруктовым салатом. Десертные вина из гевюрцтраминера сочетаются с мягкими сырами, особенно с острыми и солеными: рокфором, горгонзолой, мюнстером, выдержанными бри и камамбером. Ноты личи, цветов и пряностей гевюрцтраминеру придают природные терпены — особые компоненты эфирных масел. На аромат гевюрца сильно влияет благородная плесень botrytis cinerea. Она выделяет глицерин, который придает вину вязкость и усиливает вкусовые ощущения. В Риме гевюрцтраминер был любимым вином Цезаря.

In [17]:
thread.delete()

Мы видим, что ответ получился несколько однобоким, поскольку данные о сочетании вин и еды содержатся в текстовой базе знаний в разрозненном виде.

## Добавляем таблицу соответствий

Поскольку подбор блюда к вину является частой задачей, добавим к нашей базе знаний явную табличку соответствий блюд и вин, которая находится в файле `data/food_wine_table.md` в формате markdown.

In [18]:
with open("data/food_wine_table.md", encoding="utf-8") as f:
    food_wine = f.readlines()
fw = "".join(food_wine)

tokens = len(model.tokenize(fw))
print(f"Токенов: {tokens}, {len(fw)/tokens} chars/token")

Токенов: 12630, 3.3103721298495645 chars/token


In [19]:
printx(fw[:1000])

Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Баклажаны, запеченые с сыром | Красное вино: «среднетелые»* сухие — Гренаш (Гарнача), Санджовезе (Кьянти), Карменер, Менсия, молодые Темпранильо, легкотелое Мерло.
Баранина деликатесная (филе или каре ягненка) | Красное вино: сухие выдержанные вина из винограда Пино Нуар, Менсия, Неббиоло (в том числе элегантные выдержанные Бароло и Барбареско), Гамэ (элегантные бургундские Божоле Виляж).
Баранина пикантная: жареная, гриль, тушеная — со специями | Красные вина: сухие вина из винограда Каберне Совиньон, «ронские»** ассамбляжи Гренаш+Сира+Мурведр, французский Мальбек, немного «скругленная» Барбера, Сира (Шираз). Выдержанные вина из Санджовезе (Кьянти Классико, вина Монтальчино), Альянико, «супертосканские»*** вина, добротные Crianza Риохи. Примитиво и Зинфандель. Саперави из России.
Бефстроганов | Белые вина: выдержанные в дубе Шардоне, Пино Гриджо (лучше — из Северной Италии), Вердехо, Вермент

Видим, что табличка большая, поэтому её придётся *чанковать*. Но при этом важно чанковать табличку так, чтобы в каждом фрагмента оставался заголовок таблицы, который определяет семантику столбцов.

Отделим заголовок таблицы:

In [20]:
header = food_wine[:2]
header

['Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду\n',
 '--------|--------\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [21]:
chunk_size = 600 * 3  # approx 600 tokens * 2 char/token

s = header.copy()
uploaded_foodwine = []
for x in food_wine[2:]:
    s.append(x)
    if len("".join(s)) > chunk_size:
        id = sdk.files.upload_bytes(
            "".join(s).encode(), ttl_days=5, expiration_policy="static",
            mime_type="text/markdown",
        )
        #printx("".join(s))
        uploaded_foodwine.append(id)
        s = header.copy()
print(f"Uploaded {len(uploaded_foodwine)} table chunks")

Uploaded 22 table chunks


Теперь добавим эти фрагменты в индекс:

In [22]:
op = index.add_files_deferred(uploaded_foodwine)
xfiles = op.wait()

Посмотрим, стал ли ответ системы лучше:

In [23]:
thread = create_thread()

thread.write("Какое вино подходит к стейку?")
run = assistant.run(thread)

result = run.wait()
printx(result.text)
print_citations(result)

Выбор вина к стейку зависит от степени его прожарки и жирности. Вот несколько рекомендаций:

1. Для нежного мраморного стейка (например, филе-миньон) подойдут лёгкие и элегантные красные вина из винограда Пино Нуар, Нерелло Маскалезе, а также выдержанное и элегантное Мерло. Кроме того, можно рассмотреть «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва).

2. Для жирноватого стейка (например, рибай) с прожаркой Rare рекомендуются выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки.

3. Для прожарки Medium или WellDone к жирноватому стейку подойдут сухие и полусухие вина из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6–8 лет выдержки и выше.

------------------------


Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Солянка (первое блюдо) | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Спагетти Карбонара | Белые вина: сухие выдержанные вина из Шардоне и Вердехо. Розовые вина: среднетелые и «тельные» вина Франции, Италии, Испании, России. Красные вина:  выдержанное «округлое» Неббило, элегантные Пино Нуары, Нерелло Маскалезе. Спагетти Болоньезе | Красные вина: сухие из винограда Санджовезе, Барбера, Неро д’Авола, Негроамаро, молодые Неббило, а также международные — Каберне Совиньон, молодоватое испанское Темпранильо. Спаржа обжаренная | Белые вина: тихое сухое из сорта Шардоне, Грюнер Вельтлинер, Мюллер Тургау, Совиньон Блан легкие и не очень кислотный Рислинг, Пино Гри. Как вариант — Игристые вина: «тельное» и капельку сладковатое (брют, сухое, полусухое) белое. Стейк говяжий мраморный нежный (Филе-миньон) | Красные вина: легкие и элегантные из винограда Пино Нуар, Нерелло Маскалезе, элегантно сделанное и выдержанное Мерло. Также подойдут «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва). Стейк говяжий мраморный жирноватый (Рибай и пр.) | Красные вина: к прожарке Rare — выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки. К прожарке Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6-8 лет выдержки и выше. Суши, сашими | см. Роллы

------------------------


## Франшхук 

Франшхук
Франшхук — крупная винодельческая провинция Южной Африки. Территориально относится к региону Стелленбош, расположена в 75 км от Кейптауна — столицы ЮАР. Деревню основали беглые французы в 1688 году. Получив земельные наделы, они разбили виноградники и наладили производство напитков. Сегодня Франшхук называют винной столицей ЮАР. На карте провинции можно насчитать 11 крупных винодельческих хозяйств. Деревня Франшхук известна необычным терруаром: она расположена в продолговатой долине и с трех сторон окружена высокими горами, которые защищают виноградники от ветра, излишней влаги зимой и палящего солнца летом. При этом четвертая сторона долины открыта для ветров Атлантики. Климат здесь умеренный, а температура воздуха ниже, чем в соседних областях. Виноград созревает медленнее, чем на открытой африканской местности, что делает вино более свежим и кислотным. В Франшхуке наиболее распространены известные европейские сорта, завезенные французами в XVII веке. Здесь выращивают белые шенен блан, семильон, совиньон блан и шардоне. Среди красных популярны каберне совиньон, шираз и мерло. Значительная часть плантаций занята под пинотаж — автохтонный сорт, ставший визитной карточкой страны. Линейка традиционных вин из европейских сортов в ЮАР открывается совершенно по-новому. Сухие гранитные почвы придают напиткам явную минеральность. Белые вина приобретают золотистый оттенок и аромат южных фруктов, красные — насыщенный цвет и аромат. Оттенок может варьироваться от ярко-рубинового до глубокого фиолетового. Их часто сравнивают с бургундскими напитками. С 1992 года в долине Франшхук производят игристые вина премиального качества из шардоне и пино нуар по классическому методу. Белые вина Франшхука — прекрасный аперитив. Они подходят к легким сырам, закускам из гусиного паштета и курицы. Среди горячих блюд стоит выбрать индейку, запеченную белую рыбу с овощным гарниром или рагу. Красные южноафриканские сорта хорошо гармонируют с перчеными стейками из говядины и баранины, а также твердыми, выдержанными сырами. Пинотаж при этом считается универсальным вином. Ему подходят разнообразные блюда от изысканного филе миньон до лазаньи и пиццы с морепродуктами.

...

Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Салат Цезарь с курицей | Белые вина: сухое и полусухое из винограда Шардоне, Вердехо, Пино Гри, Шенен Блан. Также подойдут Розовые вина: тихие легкие и элегантные вина Прованса (Франция) или Северной Италии. Неплохо пойдут легкие Оранжи (особенно если курятина жарилась на мангале). Салат Цезарь с лососем | Белые вина: сухое выдержанное Шардоне, Альбариньо, Пино Гри (в стиле Альто-Адидже). Полусухие белые вина со средней интенсивностью аромата и вкуса — Семильон, Рислинг, Вердехо. Розовые вина: тихие легкие и элегантные вина Прованса (Франция) или Северной Италии. Салаты азиатские пикантные | Белые вина: сухие и полусухие из винограда Рислинг, Мускат, Гевюрцтраминер. Розовые вина: среднетелые и «тельные»  вина Франции, Италии, Испании, России. Как вариант — розовые молодые и недорогие тельные Игристые вина. К салатам с прикопченым или просто пикантным мясом — можно выбрать легкие Красные вина: Нино Нуары, Нерелло Маскалезе, выдержанное Неббиоло. Неплохо пойдут и Оранжи. Сало соленое или прикопченое | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Свинина в кисло-сладком соусе | Белые вина: Рислинг, сухой Гевюрцтраминер или аналогичный Мускат
Свинина постная, медальоны | Белые вина: тихое сухое выдержанное Шардоне, Альбариньо, Пино Гри (в стиле Альто-Адидже). Полусухие белые вина со средней интенсивностью аромата и вкуса — Семильон, Рислинг, Вердехо. Розовые вина: тихие среднетелые и «тельные» вина Франции, Италии, Испании, России. Свинина стейк жареный с луком | Красные вина: сухие и полусухие вина из винограда Гарнача (Гренаш), Мерло, Карменер, Менсия, «тельные» Пино Нуары (со всего света), российский Красностоп, Гамэ (Божоле Виляж). Сельдь с лучком (закуска) | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Для глобальных приверженцев вина — тельный, сладковатый и мощновкусный Рислинг.

------------------------


Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Рыба — стерлядь, осетр (деликатесные, прикопченые) | Русская классика — Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Для любителей вина — настоятельно рекомендую Белые вина: тихие Шардоне (бургундского стиля), сухие и полусухие слегка сладковатые Рислинги невысокой кислотности, Пино Гри (из Альто Адидже), среднетелые и среднекислотные Альбариньо и Вердехо. Как вариант — Игристые вина (брюты и сухие — белые и легчайшие розовые). Рыба Тунец | В зависимости от типа приготовления (от самого нежного к зажаренному и пикантному) — Белые вина: тихие и элегантные выдержанные Шардоне и Вионье, сухие и полусухие Рислинги, Совиньон Блан из Нового Света. Розовые вина: тихие среднетелые сухие вина Франции, Италии, Испании, России. Как вариант к пикантному — Красные вина: легкие вина из Пино Нуара, Нерелло Маскалезе, Гриньолино (Италия), «округленькие» выдержанные Неббиоло и даже Гамэ (выдержанные Божоле Виляж). Подойдут также Оранжи. Салат Греческий | Белые вино: легкие, яркие сухие вина в первую очередь — из «родного» винограда Ассиртико, но отлично также подойдут Мюллер Тургау, Грюнер Вельтллинер, Вердехо, Шенен Блан, Совиньон Блан, Мелон де Бургонь (французские Мюскаде), вина из португальского региона Винью Верде. Как вариант — Игристые вино: белые и очень легкие розовые (брют-натур, экстра-брют, брют, сухое). Салат Еврейский | Белые вино: сухие выдержанные — Шардоне, Альбариньо, Пино Гри (в стиле Альто-Адидже), Семийон, полусухой сладковатый Рислинг, Вердехо. Розовые вина: тихие сухие и полусухие среднетелые вина Франции, Италии, Испании, России. Как вариант — белые и розовые молодые и недорогие тельные Игристые вина. Салат Капрезе | Белые вина: игристые (брют, сухие), тихие вина — с ярким и свежим вкусом: молодые Шардоне, Рислинги, Мюллер Тургау, Грюнер Вельтлинер, Виура, «старосветский» Совиньон Блан, Пино Гри, Фалангина и т. д. Как вариант — Розовые вина: тихие легкие и элегантные вина Прованса (Франция) или Северной Италии, в также попробуйте Игристые вина: легкие и элегантные белые и розовые.

------------------------


## Орегон 

Орегон
Орегон — четвертый по объемам производства вина штат в США. Самый популярный сорт винограда, который выращивают здесь, — пино нуар. Вина из этого сорта отличает высокая кислотность и низкая танинность. Выращивать виноград в Орегоне начали еще в 1840-х, но датой зарождения серьезного виноделия называют 1961 год, когда любители бургундского пино нуар Ричард Соммер и Дэвид Летт решили производить похожее вино в Америке. Для этого был выбран Орегон — прохладный климат и разнообразие почв позволили добиться успеха. Уже в 1979 пино нуар из Орегона вошло в десятку вин в бургундском стиле на французской винной Олимпиаде. Его отличают более яркие ноты фруктов и более низкая кислотность. Прохладный климат Орегона хорошо подходит для выращивания таких сортов винограда, как рислинг, шардоне и гаме, но визитной карточкой штата считается пино нуар, который растет в долине Уилламетт. Менее популярны мерло, каберне совиньон, зинфандель. В Орегоне производят сухие, полусухие, а также игристые и десертные вина. Пино нуар из Орегона будет хорошо сочетаться со стейком из лосося, запеченной птицей и красным мясом, например, говядиной по-бургундски.

In [24]:
thread.delete()
assistant.delete()

## Function Calling

Предположим, мы делаем ассистента для магазина вин, который торгует винами по некоторому прайс-листу. Возьмём пример такого прайс-листа:

In [25]:
import pandas as pd

pl = pd.read_excel("data/wine-price.xlsx")
pl

,Номер артикула,название,CT,цена от 1 бутылки,от 3-х и более,регулярная цена/промо
0,56885,"0,75Л ВИНО САССИКАЙЯ КР СХ",IT,27799.000,19459.3000,NaN
1,666560,"0,75ВИНО СИЕПИ МАЗЕЙ КР СХ",IT,15999.000,11199.3000,NaN
2,533769,"0,75ВИНО ПАЛАФРЕНО КР СХ",IT,14999.004,10499.3028,NaN
3,93733,"0,75ВИНО АНТ ТИНЬЯНЕЛЛО КР СХ",IT,14499.012,10149.3084,NaN
4,644863,"0,75ВИНО ШАТО МОНРОЗ КР СХ",FR,12999.000,9099.3000,от промо цены
...,...,...,...,...,...,...
747,61418,"0,7ВИНО КАГОР ТАМ КР СЛ",RU,179.004,125.3028,от промо цены
748,615581,"0,187ВИНО ДЖАСТ МЕРЛО КР СХ",FR,149.004,104.3028,NaN
749,615582,"0,187ВИНО ДЖАСТ КБСВ КР СХ",FR,149.004,104.3028,NaN
750,83302,"0,187Л ВИНО АДАГУМ КБСВ КР СХ",RU,119.004,83.3028,NaN


Для удобства переименуем колонки:

In [26]:
pl.columns = ["Id", "Name", "Country", "Price", "WHPrice", "etc"]

Из имени извлечём кислотность вина:

In [27]:
acid_map = {"СХ": "Сухое", "СЛ": "Сладкое", "ПСХ": "Полусухое", "ПСЛ": "Полусладкое"}
pl["Acidity"] = pl["Name"].apply(
    lambda x: acid_map.get(x.split()[-1].replace("КР", ""), "")
)
pl["Acidity"].value_counts()

Сухое          614
Полусухое       68
Полусладкое     28
Сладкое         23
                19
Name: Acidity, dtype: int64

Тоже самое сделаем с цветом:

In [28]:
pl["Color"] = pl["Name"].apply(
    lambda x: (
        "Красное" if (x.split()[-1].startswith("КР") or x.split()[-2] == "КР") else ""
    )
)
pl["Color"].value_counts()

Красное    739
            13
Name: Color, dtype: int64

В итоге получилась такая таблица вин:

In [29]:
pl

,Id,Name,Country,Price,WHPrice,etc,Acidity,Color
0,56885,"0,75Л ВИНО САССИКАЙЯ КР СХ",IT,27799.000,19459.3000,NaN,Сухое,Красное
1,666560,"0,75ВИНО СИЕПИ МАЗЕЙ КР СХ",IT,15999.000,11199.3000,NaN,Сухое,Красное
2,533769,"0,75ВИНО ПАЛАФРЕНО КР СХ",IT,14999.004,10499.3028,NaN,Сухое,Красное
3,93733,"0,75ВИНО АНТ ТИНЬЯНЕЛЛО КР СХ",IT,14499.012,10149.3084,NaN,Сухое,Красное
4,644863,"0,75ВИНО ШАТО МОНРОЗ КР СХ",FR,12999.000,9099.3000,от промо цены,Сухое,Красное
...,...,...,...,...,...,...,...,...
747,61418,"0,7ВИНО КАГОР ТАМ КР СЛ",RU,179.004,125.3028,от промо цены,Сладкое,Красное
748,615581,"0,187ВИНО ДЖАСТ МЕРЛО КР СХ",FR,149.004,104.3028,NaN,Сухое,Красное
749,615582,"0,187ВИНО ДЖАСТ КБСВ КР СХ",FR,149.004,104.3028,NaN,Сухое,Красное
750,83302,"0,187Л ВИНО АДАГУМ КБСВ КР СХ",RU,119.004,83.3028,NaN,Сухое,Красное


Чтобы научиться отвечать на вопросы по прайс-листу, у нас есть несколько возможных решений:

* Попытаться закинуть прайс-лист в контекст модели с помощью RAG, как мы это ранее делали с таблицей блюд. Не очень хороший вариант, потому что модель не будет видеть всю таблицу, и не сможет отвечать на вопросы "максимальная/минимальная цена".
* Попытаться организовать трансляцию запроса не естественном языке в SQL-подобный язык. Это идеальный вариант, но его сложно сделать без ошибок без fine-tuning-а модели. 
* Извлечь из текстового запроса основные параметры того, что хочет пользователь, и затем сформировать на этой основе запрос, извлечающий данные из таблицы. Такой подход описан, например, в статье [Querying Databases with Function Calling](https://arxiv.org/html/2502.00032v1)

В любом случае, два последних варианта подразумевают использование технологии Function Calling - когда модель может сама решить, когда ей надо вызвать некоторую **внешнюю функцию**, и какие параметры нужно ей передать.

Чтобы function calling работал - нам надо сообщить LLM о доступных **инструментах**. Это можно сделать, передав с помощью JSON-схемы описание возможностей таких инструментов и их параметров.

ML SDK позволяет нам упростить function calling, и вместо JSON-схемы использовать типизированные объекты Pyton. Для извлечения параметров запроса о вине, мы создадим такой объект: 

In [30]:
from pydantic import BaseModel, Field
from typing import Optional

class SearchWinePriceList(BaseModel):
    """Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам."""

    name: str = Field(description="Название вина", default=None)
    country: str = Field(description="Страна", default=None)
    acidity: str = Field(
        description="Кислотность (сухое, полусухое, сладкое, полусладкое)", default=None
    )
    color: str = Field(description="Цвет вина (красное, белое, розовое)", default=None)
    sort_order: str = Field(
        description="Порядок выдачи (most expensive, cheapest, random, average)",
        default=None,
    )
    what_to_return: str = Field(
        description="Что вернуть (wine info или price)", default=None
    )

Теперь создадим инструмент (tool) и нового ассистента, у которого в списке инструментов будет одновременно и RAG-поиск, и function calling. Также в инструкции ассистенту пропишем, что он может использовать Function Calling.

In [31]:
price_list_search_tool = sdk.tools.function(SearchWinePriceList)

assistant = create_assistant(model, tools=[price_list_search_tool, search_tool])
thread = create_thread()

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций. Если вопрос касается конкретных вин
или цены, то используй Function Calling.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

_ = assistant.update(instruction=instruction)

Попробуем узнать самое дешевое вино из Австралии:

In [32]:
thread.write("Привет! Какое есть самое дешевое красное сухое вино из Австралии?")
run = assistant.run(thread)
res = run.wait()
res

RunResult(status=<RunStatus.TOOL_CALLS: 5>, error=None, tool_calls=ToolCallList(ToolCall(function=FunctionCall(name='SearchWinePriceList', arguments={'what_to_return': 'price', 'acidity': 'сухое', 'color': 'красное', 'sort_order': 'cheapest', 'country': 'Австралия'})),), _message=None, usage=Usage(input_text_tokens=2591, completion_tokens=39, total_tokens=2630))

Видим, что в итоге вернулся ответ, указывающий, что надо сделать Funtion Call для получения данных из таблицы. При это LLM извлекла из текста все параметры запроса.

Реализуем функцию, которая возвращает список вин по параметрам, заданным в виде объекта `SearchWinePriceList`:

In [33]:
country_map = {
    "IT": "Италия",
    "FR": "Франция",
    "ES": "Испания",
    "RU": "Россия",
    "PT": "Португалия",
    "AR": "Армения",
    "CL": "Чили",
    "AU": "Австрия",
    "GE": "Грузия",
    "ZA": "ЮАР",
    "US": "США",
    "NZ": "Новая Зеландия",
    "DE": "Германия",
    "AT": "Австрия",
    "IL": "Израиль",
    "BG": "Болгария",
    "GR": "Греция",
    "AU": "Австралия",
}

revmap = {v.lower(): k for k, v in country_map.items()}


def find_wines(req):
    x = pl.copy()
    if req.country and req.country.lower() in revmap.keys():
        x = x[x["Country"] == revmap[req.country.lower()]]
    if req.acidity:
        x = x[x["Acidity"] == req.acidity.capitalize()]
    if req.color:
        x = x[x["Color"] == req.color.capitalize()]
    if req.name:
        x = x[x["Name"].apply(lambda x: req.name.lower() in x.lower())]
    if req.sort_order and len(x)>0:
        if req.sort_order == "cheapest":
            x = x.sort_values(by="Price")
        elif req.sort_order == "most expensive":
            x = x.sort_values(by="Price", ascending=False)
        else:
            pass
    if x is None or len(x) == 0:
        return "Подходящих вин не найдено"
    return "Вот какие вина были найдены:\n" + "\n".join(
        [
            f"{z['Name']} ({country_map.get(z['Country'],'Неизвестно')}) - {z['Price']}"
            for _, z in x.head(10).iterrows()
        ]
    )


print(find_wines(SearchWinePriceList(country="Австралия", sort_order="cheapest")))

Вот какие вина были найдены:
0,75ВИНО ДЖИНДАЛИ МЕРЛО КР ПСХ (Австралия) - 499.0
0,75ВИНО ДЖИНДАЛИ КБСВ КР ПСХ (Австралия) - 499.0
0,75ВИНО ЧОЛК ХИЛЛ ШИРАЗ КР СХ (Австралия) - 509.0
0,75ВИНО ПИТ'С ПЮР ШИРАЗ КР ПСХ (Австралия) - 579.0
0,75ВИНО ПИТ'С ПЮР ПННР КР ПСХ (Австралия) - 579.0
0,75ВИНО СТАМП ДЖАМП КР СХ (Австралия) - 789.0
0,75ВИНО ЛИНД БИН50 ШИР КР ПСХ (Австралия) - 899.0
0,75ВИНО ЛЭКИ ШИРАЗ КРСХ (Австралия) - 978.996
0,75ВИНО СТЭДФАСТ ШИР КАБ КРСХ (Австралия) - 999.0
0,75ВИНО ТИРРЕЛЗ ШИР КР СХ (Австралия) - 1098.996


В ответ на Function Call нам нужно сформировать ответ от функции, и передать её обратно для обработки языковой моделью. В нашем случае функция может быть только одна, поэтому мы не проверяем название функции, а всегда запрашиваем поиск в таблице:

In [34]:
import time

if res.tool_calls:
    result = []
    for f in res.tool_calls:
        print(f" + Processing function call fn={f.function.name}")
        x = SearchWinePriceList.model_validate(f.function.arguments)
        x = find_wines(x)
        result.append({"name": f.function.name, "content": x})
    run.submit_tool_results(result)
    time.sleep(3)
    res = run.wait()
res

 + Processing function call fn=SearchWinePriceList


RunResult(status=<RunStatus.COMPLETED: 4>, error=None, tool_calls=None, _message=Message(id='fvte9m8qca8rpproir3a', parts=('Самое дешёвое красное сухое вино из Австралии в нашем магазине — это "ВИНО ЧОЛК ХИЛЛ ШИРАЗ КР СХ" по цене 509.0 рублей за бутылку объёмом 0,75 литра.',), thread_id='fvt0i0qqig1cibosk6as', created_by='ajej20rll4tifkelclga', created_at=datetime.datetime(2025, 4, 2, 10, 21, 58, 361478), labels=None, author=Author(id='fvta102e3c12iotqa9eu', role='ASSISTANT'), citations=()), usage=Usage(input_text_tokens=3268, completion_tokens=85, total_tokens=3353))

In [35]:
thread.delete()
assistant.delete()

## Релизуем агента с Function Calling

Для упрощения реализации Function Calling напишем небольшую обвязку, реализующую агента, способного искать в текствой базе и делать Function Calling. 

Функцию обработки запроса мы включим в состав класса для описания функции, назовём её `process`. Для реализации всех наших задумок также будем передавать в неё текущий `thread`:

In [36]:
class SearchWinePriceList(BaseModel):
    """Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам."""

    name: str = Field(description="Название вина", default=None)
    country: str = Field(description="Страна", default=None)
    acidity: str = Field(
        description="Кислотность (сухое, полусухое, сладкое, полусладкое)", default=None
    )
    color: str = Field(description="Цвет вина (красное, белое, розовое)", default=None)
    sort_order: str = Field(
        description="Порядок выдачи (most expensive, cheapest, random, average)",
        default=None,
    )
    what_to_return: str = Field(
        description="Что вернуть (wine info или price)", default=None
    )

    def process(self, thread):
        return find_wines(self)

Также в виде функции реализуем функциональность передачи управления оператору. В данном случае мы будем просто устанавливать некоторый глобальный флаг `handover` - это очень грубое решение и сделано для простоты, в реальной жизни надо будет подумать над более правильной логикой, поддерживающей нескольких пользователей.

In [37]:
handover = False

class Handover(BaseModel):
    """Эта функция позволяет передать диалог человеку-оператору поддержки"""

    reason: str = Field(
        description="Причина для вызова оператора", default="не указана"
    )

    def process(self, thread):
        global handover
        handover = True
        return f"Я побежала вызывать оператора, ваш {thread.id=}, причина: {self.reason}"

Также реализуем функцию добавления вин в корзину. Чтобы для каждого пользователя была своя корзина, будем привязывать её к `thread.id` с помощью словаря `carts`:

In [38]:
carts = {}


class AddToCart(BaseModel):
    """Эта функция позволяет положить или добавить вино в корзину"""

    wine_name: str = Field(
        description="Точное название вина, чтобы положить в корзину", default=None
    )
    count: int = Field(
        description="Количество бутылок вина, которое нужно положить в корзину",
        default=1,
    )

    def process(self, thread):
        if thread.id not in carts:
            carts[thread.id] = []
        carts[thread.id].append(self)
        return f"Вино {self.wine_name} добавлено в корзину, число бутылок: {self.count}"

Наконец, оформим функцию для показа корзины:

In [39]:
class ShowCart(BaseModel):
    """Эта функция позволяет показать содержимое корзины"""

    def process(self, thread):
        if thread.id not in carts or len(carts[thread.id]) == 0:
            return "Корзина пуста"
        return "В корзине находятся следующие вина:\n" + "\n".join(
            [f"{x.wine_name}, число бутылок: {x.count}" for x in carts[thread.id]]
        )

Теперь реализуем главный класс `Agent`, который будет брать на себя обработку функций. В качестве `tools` будем передавать список описанных нами ранее классов.

Также предусмотрим гибкую работу с `thread`. При запросе агента мы сможем опиционально указывать ему уже созданный `thread` для ведения переписки, либо же переписка будет вестись в созданном потоке по умолчанию. Это полезно для упрощения нашего дальнейшего кода.

In [40]:
class Agent:
    def __init__(self, assistant=None, instruction=None, search_index=None, tools=None):

        self.thread = None

        if assistant:
            self.assistant = assistant
        else:
            if tools:
                self.tools = {x.__name__: x for x in tools}
                tools = [sdk.tools.function(x) for x in tools]
            else:
                self.tools = {}
                tools = []
            if search_index:
                tools.append(sdk.tools.search_index(search_index))
            self.assistant = create_assistant(model, tools)

        if instruction:
            self.assistant.update(instruction=instruction)

    def get_thread(self, thread=None):
        if thread is not None:
            return thread
        if self.thread == None:
            self.thread = create_thread()
        return self.thread

    def __call__(self, message, thread=None):
        thread = self.get_thread(thread)
        thread.write(message)
        run = self.assistant.run(thread)
        res = run.wait()
        if res.tool_calls:
            result = []
            for f in res.tool_calls:
                print(
                    f" + Вызываем функцию {f.function.name}, args={f.function.arguments}"
                )
                fn = self.tools[f.function.name]
                obj = fn(**f.function.arguments)
                x = obj.process(thread)
                result.append({"name": f.function.name, "content": x})
            run.submit_tool_results(result)
            #time.sleep(3)
            res = run.wait()
        return res.text

    def restart(self):
        if self.thread:
            self.thread.delete()
            self.thread = sdk.threads.create(
                name="Test", ttl_days=1, expiration_policy="static"
            )

    def done(self, delete_assistant=False):
        if self.thread:
            self.thread.delete()
        if delete_assistant:
            self.assistant.delete()

Создадим нашего агента - винного сомелье. В системном промпте пропишем ему возможные варианты для вызова функции, чтобы облегчить задачу.

In [41]:
instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию SearchWinePriceList.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

wine_agent = Agent(
    instruction=instruction,
    search_index=index,
    tools=[SearchWinePriceList, Handover, AddToCart, ShowCart],
)

In [42]:
printx(wine_agent("Какое вино пьют со стейком?"))

К стейку можно порекомендовать следующие вина:

1. Для нежного мраморного стейка (Филе-миньон) подойдут лёгкие и элегантные красные вина из винограда Пино Нуар, Нерелло Маскалезе, а также выдержанное и элегантное Мерло. Также можно рассмотреть «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва).

2. Для мраморного жирноватого стейка (Рибай и пр.) в зависимости от прожарки можно выбрать:
   - Для прожарки Rare — выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки.
   - Для прожарки Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6–8 лет выдержки и выше.

In [43]:
printx(wine_agent("Какие вина Кьянти есть в продаже?"))

 + Вызываем функцию SearchWinePriceList, args={'what_to_return': 'wine info', 'name': 'Кьянти'}


В нашем магазине в продаже есть следующие вина Кьянти:

1. Вино Кверчаб Кьянти красное сухое, 0,75 л - 2499.0 руб.
2. Вино Полиц Кьянти красное сухое, 0,75 л - 1749.756 руб.
3. Вино Касал Кьянти супер красное сухое, 0,75 л - 1349.004 руб.
4. Вино Век Кант Кьянти красное сухое, 0,75 л - 1099.0 руб.
5. Вино Пределла Кьянти красное сухое, 1,5 л - 999.0 руб.
6. Вино Зонин Кьянти красное сухое, 0,75 л - 699.0 руб.
7. Вино Пределла Кьянти красное сухое, 0,75 л - 369.0 руб.

In [44]:
printx(wine_agent("Добавь в корзину Полиц Кьянти, три бутылки"))

 + Вызываем функцию AddToCart, args={'wine_name': 'Полиц Кьянти', 'count': 3.0}


Вино "Полиц Кьянти" успешно добавлено в корзину в количестве трех бутылок. Если вам нужно что-то еще или хотите просмотреть содержимое корзины, дайте знать!

In [45]:
printx(wine_agent("Ещё положи в корзину Зонин Кьянти"))

 + Вызываем функцию AddToCart, args={'wine_name': 'Зонин Кьянти', 'count': 1.0}


Вино "Зонин Кьянти" успешно добавлено в корзину. Если вам нужно что-то еще или хотите просмотреть содержимое корзины, дайте знать!

In [46]:
printx(wine_agent("Что у меня в корзине?"))

 + Вызываем функцию ShowCart, args={}


В вашей корзине находятся следующие вина:
- Полиц Кьянти (3 бутылки)
- Зонин Кьянти (1 бутылка)

In [47]:
printx(wine_agent("Вызови оператора, хочу оформить доставку!"))

 + Вызываем функцию Handover, args={'reason': 'Пользователь хочет оформить доставку'}


Ожидайте, скоро с вами свяжется оператор для оформления доставки.

## Многоагентное тестирование

Когда мы сделали такого бота, возникает вопрос, как его тестировать. Для этого возможно несколько вариантов:

* Ручное тестирование (примерно то, что мы проделали выше)
* Автоматическое тестирование на заранее заготовленном датасете диалогов, с формализованной проверкой метрик. Такое тестирование удобно проводить с помощью специализированных фреймворков, например, RAGAS.

Мы в данном примере предлагаем промежуточный по сложности подход - это **многоагентное тестирование**, при котором мы инструктируем другого агента, чтобы он побеседовал с нашим винным ассистентом в определённом ключе. В итоге у нас могут получиться различные сценарии диалога, глядя на которые можно будет определить направление доработки исходного ассистента.

In [51]:
instruction_user = """
Ты - простой человек, и тебе нужно выбрать вино в интернет-магазине для ужина.
Ты не очень много понимаешь в винах, и хочешь побольше распросить сомелье о разных вариантах.
Ты хотел бы съесть стейк, но готов изменить свой выбор еды, если поймёшь хорошее сочетание с вином. 
Говори простым языком, короткими разговорными фразами. Когда ты удовлетворём выбором, попроси 
найти самое недорогое вино данного типа, затем положи его в корзину. В конце попроси соединить тебя 
с оператором, чтобы оформить доставку. Каждое действие выполняй в одной фразой в ответ на сообщение сомелье. Никогда 
не продолжай диалог, больше, чем одной фразой. Не пиши реплики от лица сомелье или кого-то другого.
"""

user = Agent(instruction=instruction_user)

msg = "Добрый день! Поможете мне выбрать вино?"
handover = False
for i in range(10):
    printx(f"**Посетитель:** {msg}")
    msg = wine_agent(msg)
    printx(f"**Сомелье:** {msg}")
    if handover:
        break
    msg = user(msg)

**Посетитель:** Добрый день! Поможете мне выбрать вино?

**Сомелье:** Конечно, помогу! Для того чтобы подобрать наиболее подходящее вино, мне нужно знать, с каким блюдом вы планируете его сочетать. Пожалуйста, опишите блюдо или сообщите, к какому типу оно относится (например, рыбное, мясное, салат, сыр и т. д.).

**Посетитель:** Я хочу стейк, но готов поменять блюдо, если вино лучше подойдёт к чему-то другому. Посоветуй, какое вино и с каким блюдом хорошо сочетаются?

**Сомелье:** Конечно, вот несколько интересных сочетаний вина и блюд:

1. **Каре ягненка** с «среднетелыми» винами со сбалансированной кислотностью: выдержанными «супертосканскими» винами, Брунелло ди Монтальчино, Кьянти Ризерва, округлыми и сочными винами из Неро д’Авола, не слишком «тельными» винами из Сиры (Шираза), резервными винами из Риохи, добротно сделанными Гарнача и Барбера, «тельными» Пино Нуарами (из Нового Света, Австрии, Германии), созревшими и отлично сбалансированными винами Бордо и Пьемонта (в том числе Бароло и Барбареско), выдержанным Арени (Армения).

2. **Индейка (стейки, котлеты, шашлычки)** с сухим и полусухим выдержанным Шардоне, Вердехо, Верментино, Семийон, Пино Гри, Шенен Блан. Также подойдут «среднетелые» розовые вина или оранжевые.

3. **Спаржа обжаренная** с тихим сухим Шардоне, Грюнер Вельтлинер, Мюллер Тургау, Совиньон Блан, легкими и не очень кислотными Рислингами, Пино Гри. Как вариант — игристые вина, «тельное» и капельку сладковатое (брют, сухое, полусухое) белое.

4. **Жульен с грибами** с сухими винами из Пино Нуара, Менсии, Нерелло Москалезе, выдержанным и бархатистым Неббиоло. Розовые вина: среднетелые и полнотелые из винограда Гренаш, Сенсо, Темпранильо, Бобаль, Сира, или «тельные» розовые игристые.

5. **Чебурек мясной** с выдержанными красными винами из сортов Каберне Совиньон, Сира, Барбера, Лагрейн, российский Цимлянский Черный и Саперави (молодое и «яркое»).

Если у вас есть предпочтения по стилю вина или какие-то конкретные пожелания, я могу уточнить рекомендации.

**Посетитель:** А есть ли недорогие вина из перечисленных? Покажи самое бюджетное из красных вин, которые хорошо подойдут к каре ягненка.

...

**Посетитель:** «ВИНО КРИС ПИНО НУАР КР СХ» из Италии.

**Сомелье:** На основе предоставленного контекста, «ВИНО КРИС ПИНО НУАР КР СХ» из Италии относится к категории вин, изготовленных из винограда Пино Нуар. Этот сорт винограда известен своей способностью отражать терруар, в котором он выращивается, и часто используется для производства тихих и игристых вин с широким диапазоном вкусовых профилей.

Если у вас есть конкретные вопросы о характеристиках этого вина или вы хотите узнать больше о сочетании его с блюдами, пожалуйста, сообщите мне, и я постараюсь предоставить более подробную информацию.

**Посетитель:** Добавить «ВИНО КРИС ПИНО НУАР КР СХ» из Италии в корзину и соединить меня с оператором, чтобы оформить доставку.

 + Вызываем функцию AddToCart, args={'wine_name': 'ВИНО КРИС ПИНО НУАР КР СХ', 'count': 1.0}
 + Вызываем функцию Handover, args={'reason': 'Оформить доставку ВИНО КРИС ПИНО НУАР КР СХ'}


**Сомелье:** Вино "ВИНО КРИС ПИНО НУАР КР СХ" из Италии успешно добавлено в корзину. Ожидайте, скоро с вами свяжется оператор для оформления доставки.

In [52]:
user.done(delete_assistant=True)
wine_agent.done(delete_assistant=True)

## Делаем ассистента проактивным

Мы обычно воспринимаем ассистентов как вопрос-ответных ботов, которые способны отвечать на вопросы пользователей. Однако с помощью системного промпта мы можем добавить им проактивности, чтобы они проявляли инициативу и сами инициировали диалог. Например:

In [53]:
instruction = """
Ты - опытный сомелье, продающий вино в магазине. Твоя задача - отвечать на вопросы пользователя
про вина, рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина,
а также проактивно предлагать пользователю приобрести вина, отвечающие его потребностям. В ответ
на сообщение /start поинтересуйся, что нужно пользователю, предложи ему какой-то
интересный вариант сочетания еды и вине, и попытайся продать ему вино.
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию SearchWinePriceList.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

wine_agent = Agent(
    instruction=instruction,
    search_index=index,
    tools=[SearchWinePriceList, Handover, AddToCart, ShowCart],
)

In [54]:
printx(wine_agent('/start'))

Здравствуйте! Я ваш виртуальный сомелье. Чем могу помочь?

Если вы не знаете, какое вино выбрать, я могу предложить несколько интересных вариантов. Например, в регионе Элим в ЮАР производят насыщенное терпким шираз, который прекрасно сочетается с блюдами из говядины, дичью и выдержанными сырами. А в Калифорнии делают каберне совиньон, который идеально подходит к пеппер-стейку.

Есть ли у вас предпочтения по стилю вина или блюду, к которому вы хотите подобрать напиток? Или, возможно, вас интересует что-то конкретное из вин определённого региона?

In [55]:
printx(wine_agent('Сколько стоит самый дорогой шираз?'))

 + Вызываем функцию SearchWinePriceList, args={'what_to_return': 'price', 'sort_order': 'most expensive', 'name': 'Шираз'}


Самым дорогим из доступных в нашем магазине вин сорта Шираз является вино "Мантра Шираз" из России, его цена составляет 1899 рублей за бутылку объемом 0,75 литра.

## Делаем винного ассистента в телеграме

Для демонстрации возможностей бота мы реализуем такого ассистента в виде телеграм-бота. Конечно, для реализации полноценного телеграм-бота необходимо использовать виртуальную машину и режим webhooks, но в нашем случае мы ограничимся режимом поллинга, и запустим бота прямо в Datasphere.

> Прежде, чем запускать код ниже, необходимо создать чат-бота, пообщавшись с [@botfather](http://t.me/botfather), и разместить его секрет в виде секрета в Datasphere.

Для начала установим необходимую библиотеку:

In [89]:
%pip install --quiet telebot

I0000 00:00:1743586576.365735    7698 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [56]:
import telebot

telegram_token = os.environ["tg_token"]

bot = telebot.TeleBot(telegram_token)

threads = {}


def get_thread(chat_id):
    if chat_id in threads.keys():
        return threads[chat_id]
    t = create_thread()
    print(f"New thread {t.id=} created")
    threads[chat_id] = t
    return t


# Обработчик команды /start
@bot.message_handler(commands=["start"])
def start(message):
    t = get_thread(message.chat.id)
    print(f"Starting on thread {t.id=}, msg={message.text}")
    ans = wine_agent(message.text, thread=t)
    bot.send_message(message.chat.id, ans)


# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    t = get_thread(message.chat.id)
    print(f"Answering on thread {t.id=}, msg={message.text}")
    answer = wine_agent(message.text, thread=t)
    bot.send_message(message.chat.id, answer)


# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

Бот готов к работе


## Удаляем лишнее

В заключение удалим созданные ресурсы:

In [57]:
wine_agent.done(delete_assistant=True)
index.delete()
for f in df["Uploaded"]:
    f.delete()
for f in uploaded_foodwine:
    f.delete()

Если вы хотите на всякий случай удалить другие оставшиеся ресурсы, то можете использовать код ниже - он удалит все переписки, файлы, индексы и ассистенты. 

**ВНИМАНИЕ**: Не выполняйте этот код, если у вас есть другие проекты с ассистентами в облаке!

In [ ]:
for thread in sdk.threads.list():
    print(f" + deleting thread id={thread.id}",end="")
    try:
        thread.delete()
    except:
        print(" ! Error",end="")
    print()
        
for assistant in sdk.assistants.list():
    print(f" + deleting assistant id={assistant.id}")
    assistant.delete()

In [97]:
from tqdm.auto import tqdm

for index in sdk.search_indexes.list():
    print(f" + deleting index id={index.id}")
    index.delete()
    
print(" + Deleting files")
for file in tqdm(sdk.files.list()):
    # print(f" + deleting file id={file.id}")
    file.delete()

 + Deleting files


0it [00:00, ?it/s]
